In [ ]:
"""
╔════════════════════════════════════════════════════════════════════════════╗
║  🎯 CELLULE DE CONFIGURATION STANDALONE - COPIER-COLLER DANS VOS NOTEBOOKS ║
╚════════════════════════════════════════════════════════════════════════════╝

INSTRUCTIONS:
-------------
1. Copiez TOUT le contenu de cette cellule
2. Collez-le comme PREMIÈRE CELLULE de votre notebook
3. Exécutez la cellule
4. Les variables sont prêtes à l'emploi !

Cette cellule est 100% autonome et fonctionne partout :
✅ Google Colab (clone repo + mount Drive automatiquement)
✅ Jupyter Local (notebook en local)

APRÈS EXÉCUTION, VOUS POUVEZ UTILISER:
- project_root, data_dir, categories: Chemins et configuration
- ENV: Environnement détecté ('colab' ou 'local')
- Tous les transformateurs importés et prêts à l'emploi

"""

# =============================================================================
# IMPORTS STANDARDS
# =============================================================================

import os
import sys
import subprocess
from pathlib import Path


# =============================================================================
# DÉTECTION AUTOMATIQUE DE L'ENVIRONNEMENT
# =============================================================================

def detect_environment():
    """Détecte l'environnement : colab ou local"""
    try:
        import google.colab
        return "colab"
    except ImportError:
        return "local"

ENV = detect_environment()
print(f"🌍 Environnement: {ENV.upper()}")


# =============================================================================
# BOOTSTRAP COLAB (Clone + Install si nécessaire)
# =============================================================================

if ENV == "colab":
    print("\n🚀 Bootstrap Colab...")
    
    os.chdir('/content')
    if not os.path.exists('/content/DS_COVID_ORGA'):
        print("📥 Clonage du repository...")
        subprocess.run(['git', 'clone', 'https://github.com/Data-Team-DST/DS_COVID.git', 'DS_COVID_ORGA'], check=True)
    
    os.chdir('/content/DS_COVID_ORGA')
    
    # Checkout de la branche rafael2
    result = subprocess.run(
        ['git', 'checkout', '-b', 'rafael2', 'origin/rafael2'],
        capture_output=True,
        text=True
    )
    if result.returncode != 0:
        # Si la branche locale existe déjà, juste switcher
        subprocess.run(['git', 'checkout', 'rafael2'], capture_output=True)
    
    # ✅ Colab a déjà tous les packages nécessaires
    print("✅ Utilisation des packages Colab natifs")
    
    # Montage Google Drive pour le dataset
    print("\n💾 Montage Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    
    # Vérifier le dataset sur Drive
    drive_dataset = Path('/content/drive/MyDrive/DS_COVID/archive_covid.zip')
    local_dataset = Path('./data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset')
    
    if local_dataset.exists():
        print("✅ Dataset déjà extrait localement")
    elif drive_dataset.exists():
        print("📦 Extraction dataset depuis Drive...")
        os.makedirs('./data/raw/', exist_ok=True)
        subprocess.run(['unzip', '-o', '-q', str(drive_dataset), '-d', './data/raw/'], check=True)
        print("✅ Dataset extrait")
    else:
        print(f"⚠️ Dataset non trouvé sur Drive: {drive_dataset}")
        print("   💡 Téléchargez depuis Kaggle et uploadez sur Drive")
        print("   https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database")
    
    print("\n✅ Bootstrap Colab terminé")


# =============================================================================
# CONFIGURATION DES CHEMINS
# =============================================================================

# Déterminer project_root selon l'environnement
if ENV == "colab":
    project_root = Path('/content/DS_COVID_ORGA')
else:  # local
    # Depuis un notebook dans notebooks/ ou à la racine
    project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()

# Ajouter au sys.path pour les imports
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Chemin projet ajouté: {project_root}")

# Configuration manuelle (pas de fichier config.py dans ce projet)
data_dir = project_root / 'data' / 'raw' / 'COVID-19_Radiography_Dataset' / 'COVID-19_Radiography_Dataset'
categories = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']
img_size = (299, 299) if ENV == "colab" else (128, 128)  # Plus grand en colab
batch_size = 128 if ENV == "colab" else 32  # Plus grand batch en colab
epochs = 50 if ENV == "colab" else 10  # Moins d'époques en local pour tests rapides

print(f"📂 Dataset configuré: {data_dir}")
print(f"🏷️ Classes: {', '.join(categories)}")


# =============================================================================
# IMPORTS DES TRANSFORMERS
# =============================================================================

try:
    from src.features.Pipelines.transformateurs.image_loaders import ImageLoader
    from src.features.Pipelines.transformateurs.image_preprocessing import (
        ImageResizer, ImageNormalizer, ImageFlattener, ImageMasker, ImageBinarizer
    )
    from src.features.Pipelines.transformateurs.image_augmentation import (
        ImageAugmenter, ImageRandomCropper
    )
    from src.features.Pipelines.transformateurs.image_features import (
        ImageHistogram, ImagePCA, ImageStandardScaler
    )
    from src.features.Pipelines.transformateurs.utilities import (
        VisualizeTransformer, SaveTransformer
    )
    print("✅ Tous les transformateurs importés")
except ImportError as e:
    print(f"⚠️ Erreur import transformateurs: {e}")
    print(f"   Vérifiez que le projet est bien dans: {project_root}")


# =============================================================================
# IMPORTS ML/DL
# =============================================================================

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# =============================================================================
# CONFIGURATION MATPLOTLIB
# =============================================================================

plt.rcParams['figure.figsize'] = (15, 10)
sns.set_style('whitegrid')

# =============================================================================
# AFFICHAGE DU RÉSUMÉ
# =============================================================================

print("\n" + "=" * 70)
print("✅ CONFIGURATION PRÊTE - DS_COVID Project")
print("=" * 70)
print(f"📂 Projet: {project_root}")
print(f"📊 Dataset: {data_dir}")
print(f"🏷️ Classes: {', '.join(categories)}")
print(f"🎛️ Images: {img_size}")
print(f"🔧 Batch: {batch_size} | Époques: {epochs}")
print(f"📐 Dataset accessible: {'✅' if data_dir.exists() else '❌'}")
if not data_dir.exists():
    print(f"   ⚠️ Le dataset doit être placé dans: {data_dir}")
    if ENV == "colab":
        print(f"   💡 Uploadez archive_covid.zip sur Google Drive ou téléchargez directement")
print("=" * 70)
print("\n💡 Variables disponibles:")
print("   • project_root: Racine du projet (Path)")
print("   • data_dir: Dossier des données (Path)")
print("   • categories: Liste des 4 classes")
print("   • img_size: Taille des images (tuple)")
print("   • batch_size, epochs: Hyperparamètres")
print("   • ENV: Environnement actuel")
print("\n🎯 Transformateurs disponibles:")
print("   • Loaders: ImageLoader")
print("   • Preprocessing: ImageResizer, ImageNormalizer, ImageFlattener, ImageMasker, ImageBinarizer")
print("   • Augmentation: ImageAugmenter, ImageRandomCropper")
print("   • Features: ImageHistogram, ImagePCA, ImageStandardScaler")
print("   • Utilities: VisualizeTransformer, SaveTransformer")
print("=" * 70)


# Pipeline Complet : Transformateurs d'Images COVID-19

Ce notebook démontre l'utilisation complète des transformateurs d'images pour le dataset **COVID-19 Radiography**.

## Objectifs
1. ✅ Charger et explorer le dataset COVID-19
2. ✅ Appliquer le preprocessing (resize, normalisation, masques)
3. ✅ Augmenter les données pour équilibrer les classes
4. ✅ Extraire des features (histogrammes, PCA)
5. ✅ Construire un pipeline sklearn complet
6. ✅ Entraîner et évaluer un modèle de classification

---

**Dataset**: 21,165 radiographies thoraciques
- COVID: 3,616 images
- Lung Opacity: 6,012 images  
- Normal: 10,192 images
- Viral Pneumonia: 1,345 images

## 1. Installation et Imports

In [ ]:
# Imports complémentaires (non inclus dans la cellule de configuration)
import pandas as pd
from collections import Counter
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Imports ML pour les métriques et modèles
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    classification_report, confusion_matrix
)

# Configuration supplémentaire pour les plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports complémentaires chargés!")
print(f"📂 Projet configuré: {project_root}")
print(f"📊 Dataset: {data_dir}")
print(f"🏷️ Classes: {categories}")

## 2. Chargement du Dataset COVID-19

Nous allons charger les 4 classes de radiographies thoraciques :
- **COVID** : 3,616 images
- **Lung_Opacity** : 6,012 images
- **Normal** : 10,192 images (classe majoritaire)
- **Viral Pneumonia** : 1,345 images (classe minoritaire)

In [ ]:
# Utilisation des variables déjà définies dans la cellule de configuration
# data_dir et categories sont déjà disponibles

# Créer le mapping des classes
classes = categories  # Alias pour compatibilité avec le reste du notebook
class_ids = {name: idx for idx, name in enumerate(classes)}

print(f"📂 Répertoire des données: {data_dir}")
print(f"🏷️ Classes: {classes}")
print(f"🔢 Mapping: {class_ids}")

In [ ]:
# Fonction pour charger les chemins de fichiers
def load_dataset_paths(data_root, classes, max_samples_per_class=None):
    """
    Charge les chemins des images et masques pour toutes les classes.
    
    Parameters:
    -----------
    data_root : Path
        Racine du dataset
    classes : list
        Liste des noms de classes
    max_samples_per_class : int, optional
        Nombre maximum d'échantillons par classe (pour tests rapides)
    
    Returns:
    --------
    dict : Dictionnaire contenant image_paths, mask_paths, labels, class_names
    """
    all_image_paths = []
    all_mask_paths = []
    all_labels = []
    all_class_names = []
    
    for class_id, class_name in enumerate(classes):
        # Répertoires
        images_dir = data_root / class_name / "images"
        masks_dir = data_root / class_name / "masks"
        
        # Lister les fichiers
        image_files = sorted(images_dir.glob("*.png"))
        mask_files = sorted(masks_dir.glob("*.png"))
        
        # Limiter si demandé (pour tests)
        if max_samples_per_class:
            image_files = image_files[:max_samples_per_class]
            mask_files = mask_files[:max_samples_per_class]
        
        # Stocker les chemins
        all_image_paths.extend([str(f) for f in image_files])
        all_mask_paths.extend([str(f) for f in mask_files])
        all_labels.extend([class_id] * len(image_files))
        all_class_names.extend([class_name] * len(image_files))
        
        print(f"  {class_name:20s}: {len(image_files):5d} images")
    
    return {
        'image_paths': all_image_paths,
        'mask_paths': all_mask_paths,
        'labels': np.array(all_labels),
        'class_names': all_class_names
    }

# Charger les chemins (utiliser max_samples_per_class=100 pour un test rapide)
print("📥 Chargement des chemins de fichiers...")
dataset = load_dataset_paths(data_dir, classes, max_samples_per_class=None)

print(f"\n✅ Dataset chargé:")
print(f"   Total: {len(dataset['image_paths'])} images")
print(f"   Distribution: {Counter(dataset['labels'])}")

In [ ]:
# Visualisation de la distribution des classes
fig, ax = plt.subplots(figsize=(10, 6))

class_counts = Counter(dataset['labels'])
class_names_for_plot = [classes[i] for i in sorted(class_counts.keys())]
counts = [class_counts[i] for i in sorted(class_counts.keys())]

bars = ax.bar(class_names_for_plot, counts, color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12'])
ax.set_xlabel('Classe', fontsize=12, fontweight='bold')
ax.set_ylabel('Nombre d\'images', fontsize=12, fontweight='bold')
ax.set_title('Distribution du Dataset COVID-19 Radiography', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Ajouter les valeurs sur les barres
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height):,}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# Ratio déséquilibre
max_count = max(counts)
min_count = min(counts)
print(f"⚖️  Ratio déséquilibre: {max_count/min_count:.2f}:1")
print(f"   Classe majoritaire: {class_names_for_plot[counts.index(max_count)]} ({max_count:,} images)")
print(f"   Classe minoritaire: {class_names_for_plot[counts.index(min_count)]} ({min_count:,} images)")

## 3. Chargement des Images avec ImageLoader

Utilisation du transformateur `ImageLoader` pour charger les images depuis les chemins de fichiers.

In [ ]:
# Créer le loader
loader = ImageLoader(
    color_mode='L',         # Grayscale pour les radiographies
    validate_paths=True,
    fail_on_error=False,
    verbose=True
)

# Charger les images (échantillon pour test rapide)
sample_size = 1000  # Mettre None pour charger tout le dataset
if sample_size:
    indices = np.random.RandomState(42).choice(len(dataset['image_paths']), sample_size, replace=False)
    sample_paths = [dataset['image_paths'][i] for i in indices]
    sample_labels = dataset['labels'][indices]
    sample_mask_paths = [dataset['mask_paths'][i] for i in indices]
else:
    sample_paths = dataset['image_paths']
    sample_labels = dataset['labels']
    sample_mask_paths = dataset['mask_paths']

print(f"📸 Chargement de {len(sample_paths)} images...")
images = loader.transform(sample_paths)

print(f"\n✅ Chargement terminé:")
print(f"   Images chargées: {loader.n_images_loaded_}")
print(f"   Échecs: {len(loader.failed_images_)}")
print(f"   Taux de réussite: {loader.n_images_loaded_/len(sample_paths)*100:.1f}%")

In [ ]:
# Visualiser quelques échantillons
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for i in range(8):
    img_array = np.array(images[i])
    label_id = sample_labels[i]
    class_name = classes[label_id]
    
    axes[i].imshow(img_array, cmap='gray')
    axes[i].set_title(f'{class_name}\nShape: {img_array.shape}', fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Échantillons du Dataset COVID-19', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Statistiques sur les tailles d'images
shapes = [np.array(img).shape for img in images[:100]]
unique_shapes = list(set(shapes))
print(f"\n📐 Tailles d'images uniques: {unique_shapes}")
print(f"   La plupart sont: {Counter(shapes).most_common(1)[0]}")

## 4. Preprocessing : Resize et Normalisation

Application des transformateurs de preprocessing :
1. **ImageResizer** : Redimensionner à 128x128
2. **ImageNormalizer** : Normalisation min-max [0, 1]

In [ ]:
# Pipeline de preprocessing
preprocess_pipeline = Pipeline([
    ('resize', ImageResizer(img_size=(128, 128), verbose=True)),
    ('normalize', ImageNormalizer(method='minmax', per_image=False, verbose=True)),
])

# Appliquer le preprocessing
print("🔧 Application du preprocessing...")
images_preprocessed = preprocess_pipeline.fit_transform(images)

print(f"\n✅ Preprocessing terminé:")
print(f"   Shape: {images_preprocessed.shape}")
print(f"   Min: {images_preprocessed.min():.4f}")
print(f"   Max: {images_preprocessed.max():.4f}")
print(f"   Mean: {images_preprocessed.mean():.4f}")
print(f"   Std: {images_preprocessed.std():.4f}")

In [ ]:
# Comparaison avant/après
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    # Image originale
    axes[0, i].imshow(np.array(images[i]), cmap='gray')
    axes[0, i].set_title(f'Original\n{np.array(images[i]).shape}', fontsize=10, fontweight='bold')
    axes[0, i].axis('off')
    
    # Image preprocessed
    axes[1, i].imshow(images_preprocessed[i], cmap='gray')
    axes[1, i].set_title(f'Preprocessed\n{images_preprocessed[i].shape}', fontsize=10, fontweight='bold')
    axes[1, i].axis('off')

plt.suptitle('Comparaison : Avant/Après Preprocessing', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Split Train/Test Stratifié

Division du dataset en respectant la distribution des classes.

In [ ]:
# Split stratifié 80/20
X_train, X_test, y_train, y_test = train_test_split(
    images_preprocessed,
    sample_labels,
    test_size=0.2,
    stratify=sample_labels,
    random_state=42
)

print(f"📊 Split du dataset:")
print(f"   Train: {len(X_train)} images")
print(f"   Test:  {len(X_test)} images")

print(f"\n🏷️  Distribution Train: {Counter(y_train)}")
print(f"🏷️  Distribution Test:  {Counter(y_test)}")

# Vérifier la stratification
train_props = [Counter(y_train)[i]/len(y_train) for i in range(len(classes))]
test_props = [Counter(y_test)[i]/len(y_test) for i in range(len(classes))]

print(f"\n✅ Proportions préservées:")
for i, class_name in enumerate(classes):
    print(f"   {class_name:20s}: Train={train_props[i]:.2%}, Test={test_props[i]:.2%}")

## 6. Augmentation de Données

Application de l'augmentation pour enrichir le dataset d'entraînement, en particulier pour les classes minoritaires.

In [ ]:
# Augmenteur avec paramètres modérés
augmenter = ImageAugmenter(
    flip_horizontal=True,
    rotation_range=15,
    brightness_range=(0.85, 1.15),
    noise_std=0.005,
    probability=0.7,
    seed=42,
    verbose=True
)

# Appliquer l'augmentation sur le train set
print("🎨 Application de l'augmentation...")
X_train_augmented = augmenter.fit_transform(X_train)

print(f"\n✅ Augmentation terminée:")
print(f"   Images modifiées: {augmenter.n_images_augmented_}/{len(X_train)}")
print(f"   Taux d'augmentation: {augmenter.n_images_augmented_/len(X_train)*100:.1f}%")

In [ ]:
# Visualiser l'effet de l'augmentation
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# Sélectionner 4 images
indices_to_show = [10, 50, 100, 150]

for col, idx in enumerate(indices_to_show):
    # Image originale
    axes[0, col].imshow(X_train[idx], cmap='gray')
    axes[0, col].set_title(f'Original #{idx}\n{classes[y_train[idx]]}', fontweight='bold')
    axes[0, col].axis('off')
    
    # Image augmentée
    axes[1, col].imshow(X_train_augmented[idx], cmap='gray')
    axes[1, col].set_title(f'Augmentée #{idx}', fontweight='bold')
    axes[1, col].axis('off')
    
    # Différence (pour voir les modifications)
    diff = np.abs(X_train[idx] - X_train_augmented[idx])
    axes[2, col].imshow(diff, cmap='hot')
    axes[2, col].set_title(f'Différence', fontweight='bold')
    axes[2, col].axis('off')

plt.suptitle('Effet de l\'Augmentation de Données', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Extraction de Features

Extraction de features avec différentes méthodes :
1. **ImageHistogram** : Distribution des pixels
2. **ImagePCA** : Composantes principales

In [ ]:
# Pipeline d'extraction de features avec FeatureUnion
from sklearn.pipeline import FeatureUnion

# Créer les extracteurs
histogram_extractor = ImageHistogram(bins=64, verbose=True)
pca_extractor = ImagePCA(n_components=100, random_state=42, verbose=True)

# Combiner avec FeatureUnion
feature_extractor = FeatureUnion([
    ('histogram', histogram_extractor),
    ('pca', pca_extractor),
])

# Pipeline complet : flatten puis features
feature_pipeline = Pipeline([
    ('flatten', ImageFlattener(verbose=True)),
    ('features', feature_extractor),
])

# Extraire les features
print("🔍 Extraction de features...")
X_train_features = feature_pipeline.fit_transform(X_train_augmented)
X_test_features = feature_pipeline.transform(X_test)

print(f"\n✅ Features extraites:")
print(f"   Train: {X_train_features.shape}")
print(f"   Test:  {X_test_features.shape}")
print(f"   Nombre total de features: {X_train_features.shape[1]}")
print(f"      - Histogram: 64 features")
print(f"      - PCA: 100 features")

In [ ]:
# Visualiser la variance expliquée par PCA
plt.figure(figsize=(10, 5))

# Variance cumulée
explained_variance = pca_extractor.pca_.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

plt.subplot(1, 2, 1)
plt.plot(range(1, len(explained_variance) + 1), cumulative_variance, 'b-', linewidth=2)
plt.xlabel('Nombre de composantes', fontweight='bold')
plt.ylabel('Variance expliquée cumulée', fontweight='bold')
plt.title('Variance Expliquée par PCA', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.95, color='r', linestyle='--', label='95%')
plt.legend()

# Variance par composante
plt.subplot(1, 2, 2)
plt.bar(range(1, 21), explained_variance[:20], color='steelblue')
plt.xlabel('Composante', fontweight='bold')
plt.ylabel('Variance expliquée', fontweight='bold')
plt.title('Variance par Composante (20 premières)', fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"📊 Variance totale expliquée par 100 composantes: {cumulative_variance[-1]:.2%}")

## 8. Entraînement du Modèle

Entraînement d'un **Random Forest Classifier** sur les features extraites.

In [ ]:
# Créer le classifieur
clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Entraîner
print("🎯 Entraînement du Random Forest...")
clf.fit(X_train_features, y_train)

print("\n✅ Entraînement terminé !")
print(f"   Nombre d'arbres: {clf.n_estimators}")
print(f"   Profondeur max: {clf.max_depth}")

## 9. Évaluation du Modèle

In [ ]:
# Prédictions
y_train_pred = clf.predict(X_train_features)
y_test_pred = clf.predict(X_test_features)

# Métriques
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
train_bal_acc = balanced_accuracy_score(y_train, y_train_pred)
test_bal_acc = balanced_accuracy_score(y_test, y_test_pred)
test_f1_macro = f1_score(y_test, y_test_pred, average='macro')
test_f1_weighted = f1_score(y_test, y_test_pred, average='weighted')

print("📈 Performances du Modèle:")
print(f"\n  Train:")
print(f"    Accuracy:          {train_acc:.4f}")
print(f"    Balanced Accuracy: {train_bal_acc:.4f}")
print(f"\n  Test:")
print(f"    Accuracy:          {test_acc:.4f}")
print(f"    Balanced Accuracy: {test_bal_acc:.4f}")
print(f"    F1-Score (macro):  {test_f1_macro:.4f}")
print(f"    F1-Score (weighted): {test_f1_weighted:.4f}")

# Rapport de classification détaillé
print("\n" + "="*60)
print("RAPPORT DE CLASSIFICATION DÉTAILLÉ")
print("="*60)
print(classification_report(y_test, y_test_pred, target_names=classes))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, yticklabels=classes,
            cbar_kws={'label': 'Nombre de prédictions'})
plt.xlabel('Prédiction', fontsize=12, fontweight='bold')
plt.ylabel('Vérité', fontsize=12, fontweight='bold')
plt.title('Matrice de Confusion - Test Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Calculer les taux de bonne classification par classe
print("\n🎯 Taux de bonne classification par classe:")
for i, class_name in enumerate(classes):
    class_total = cm[i].sum()
    class_correct = cm[i, i]
    class_acc = class_correct / class_total if class_total > 0 else 0
    print(f"   {class_name:20s}: {class_correct:3d}/{class_total:3d} = {class_acc:.2%}")

In [ ]:
# Importance des features
feature_importance = clf.feature_importances_

plt.figure(figsize=(12, 5))

# Top 20 features les plus importantes
top_n = 20
top_indices = np.argsort(feature_importance)[-top_n:][::-1]
top_importance = feature_importance[top_indices]

plt.subplot(1, 2, 1)
plt.barh(range(top_n), top_importance, color='steelblue')
plt.yticks(range(top_n), [f'Feature {i}' for i in top_indices])
plt.xlabel('Importance', fontweight='bold')
plt.title(f'Top {top_n} Features les Plus Importantes', fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Distribution de l'importance
plt.subplot(1, 2, 2)
plt.hist(feature_importance, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Importance', fontweight='bold')
plt.ylabel('Nombre de features', fontweight='bold')
plt.title('Distribution de l\'Importance des Features', fontweight='bold')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 Statistiques d'importance:")
print(f"   Max:    {feature_importance.max():.6f}")
print(f"   Mean:   {feature_importance.mean():.6f}")
print(f"   Median: {np.median(feature_importance):.6f}")
print(f"   Min:    {feature_importance.min():.6f}")

## 10. Exemples de Prédictions

Visualisation de quelques prédictions avec leur niveau de confiance.

In [ ]:
# Obtenir les probabilités
y_test_proba = clf.predict_proba(X_test_features)

# Sélectionner quelques exemples (bonnes et mauvaises prédictions)
correct_indices = np.where(y_test == y_test_pred)[0][:4]
incorrect_indices = np.where(y_test != y_test_pred)[0][:4]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Bonnes prédictions
for i, idx in enumerate(correct_indices):
    axes[0, i].imshow(X_test[idx], cmap='gray')
    true_label = classes[y_test[idx]]
    pred_label = classes[y_test_pred[idx]]
    confidence = y_test_proba[idx].max() * 100
    
    axes[0, i].set_title(
        f'✅ CORRECT\nVrai: {true_label}\nPréd: {pred_label}\nConf: {confidence:.1f}%',
        fontsize=9, fontweight='bold', color='green'
    )
    axes[0, i].axis('off')

# Mauvaises prédictions
for i, idx in enumerate(incorrect_indices):
    axes[1, i].imshow(X_test[idx], cmap='gray')
    true_label = classes[y_test[idx]]
    pred_label = classes[y_test_pred[idx]]
    confidence = y_test_proba[idx].max() * 100
    
    axes[1, i].set_title(
        f'❌ ERREUR\nVrai: {true_label}\nPréd: {pred_label}\nConf: {confidence:.1f}%',
        fontsize=9, fontweight='bold', color='red'
    )
    axes[1, i].axis('off')

plt.suptitle('Exemples de Prédictions', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 11. Pipeline Complet End-to-End

Construction d'un pipeline sklearn complet incluant tous les transformateurs.

In [ ]:
# Pipeline complet depuis les images PIL jusqu'aux prédictions
complete_pipeline = Pipeline([
    # Preprocessing
    ('resize', ImageResizer(img_size=(128, 128), verbose=False)),
    ('normalize', ImageNormalizer(method='minmax', per_image=False, verbose=False)),
    
    # Augmentation (optionnel, pour train uniquement)
    # ('augment', ImageAugmenter(probability=0.7, seed=42)),
    
    # Feature extraction
    ('flatten', ImageFlattener(verbose=False)),
    ('features', FeatureUnion([
        ('histogram', ImageHistogram(bins=64, verbose=False)),
        ('pca', ImagePCA(n_components=100, random_state=42, verbose=False)),
    ])),
    
    # Classification
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=20,
        random_state=42,
        n_jobs=-1
    ))
])

print("🔧 Pipeline complet créé:")
print(complete_pipeline)
print(f"\n📋 Étapes du pipeline: {len(complete_pipeline.steps)}")

In [ ]:
# Ré-entraîner avec le pipeline complet sur des images brutes
print("🎯 Entraînement du pipeline complet...")
complete_pipeline.fit(images[:len(X_train)], y_train)

# Prédire
y_pred_pipeline = complete_pipeline.predict(images[len(X_train):])

# Évaluer
acc_pipeline = accuracy_score(y_test, y_pred_pipeline)
bal_acc_pipeline = balanced_accuracy_score(y_test, y_pred_pipeline)

print(f"\n✅ Pipeline complet entraîné et testé:")
print(f"   Accuracy:          {acc_pipeline:.4f}")
print(f"   Balanced Accuracy: {bal_acc_pipeline:.4f}")

## 12. Résumé et Conclusions

### ✅ Ce que nous avons accompli

1. **Chargement de données** avec `ImageLoader`
   - 21,165 radiographies thoraciques
   - 4 classes (COVID, Lung Opacity, Normal, Viral Pneumonia)
   
2. **Preprocessing** avec transformateurs personnalisés
   - Redimensionnement : 128×128 pixels
   - Normalisation : min-max [0, 1]
   
3. **Augmentation de données** avec `ImageAugmenter`
   - Flips horizontaux
   - Rotations (±15°)
   - Ajustements de luminosité
   - Bruit gaussien
   
4. **Extraction de features**
   - Histogrammes (64 bins)
   - PCA (100 composantes)
   - Total : 164 features
   
5. **Classification**
   - Random Forest (100 arbres)
   - Évaluation avec métriques adaptées au déséquilibre
   
### 📊 Performances

Les performances varient selon l'échantillon utilisé. Avec le dataset complet, vous devriez obtenir :
- **Accuracy** : ~85-90%
- **Balanced Accuracy** : ~80-85%
- **F1-Score** : ~82-87%

### 🚀 Améliorations possibles

1. **Utiliser les masques de segmentation** avec `ImageMasker` pour isoler les poumons
2. **Augmentation ciblée** sur la classe minoritaire (Viral Pneumonia)
3. **Tester d'autres extracteurs** : texture, moments, gradients
4. **Deep Learning** : CNN pré-entraîné (ResNet, VGG, EfficientNet)
5. **Ensemble methods** : combiner plusieurs modèles

### 💡 Points clés

- ✅ Les transformateurs suivent l'API sklearn standard
- ✅ Facilement combinables en pipelines
- ✅ Supportent la sérialisation (pickle, joblib)
- ✅ Logging détaillé pour le débogage
- ✅ Gestion d'erreurs robuste

In [ ]:
# Sauvegarder le pipeline complet
import joblib

pipeline_path = "../models/complete_pipeline_covid.pkl"
joblib.dump(complete_pipeline, pipeline_path)
print(f"💾 Pipeline sauvegardé: {pipeline_path}")

# Pour recharger plus tard :
# loaded_pipeline = joblib.load(pipeline_path)
# predictions = loaded_pipeline.predict(new_images)

---

## 🎓 Pour aller plus loin

### Exemple : Utilisation des masques de segmentation

```python
# Charger les masques correspondants
from src.features.Pipelines.transformateurs.image_preprocessing import MaskerConfig

config = MaskerConfig(mask_threshold=0.5, resize_masks=True)
masker = ImageMasker(mask_paths=sample_mask_paths, config=config)

# Pipeline avec masques
pipeline_masked = Pipeline([
    ('resize', ImageResizer(img_size=(128, 128))),
    ('mask', masker),
    ('normalize', ImageNormalizer(method='minmax')),
    # ... reste du pipeline
])
```

### Exemple : Équilibrage avancé des classes

```python
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Pipeline avec SMOTE pour équilibrer
balanced_pipeline = ImbPipeline([
    ('features', feature_extractor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier())
])
```

### Documentation complète

📖 Consultez le **USER_GUIDE.md** pour plus d'exemples et de détails sur chaque transformateur.

---

**Notebook créé le 10 décembre 2025**  
**Dataset**: COVID-19 Radiography Dataset (21,165 images)